In [ ]:
library(ggplot2)


intensity = 2^(-8:4)

flopsmax = 16 # GB/s
        
memband <- data.frame(intensity, intensity * 16 )
colnames(memband) <- c("intesnity","FLOPS")

aplot <- ggplot(memband, aes(x=intensity, y=FLOPS, color="intensity")) + geom_point() +
            geom_line(data=amdahl, aes(x=cores,y=cores, color="Ideal")) +
            scale_color_manual(values = c("Speedup" = 'black','Ideal' = 'blue'))

print(aplot)

